In [17]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import PySAM.Pvwattsv8 as pv
import numpy_financial as npf

# Inicializar app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Parámetros por sitio
site_params = {
    "calama_simulado.csv": {"tilt": 22.5, "azimuth": 0, "lat": -22.466, "lon": -68.933, "tz": -4, "elev": 2260},
    "salvador_simulado.csv": {"tilt": 26.2, "azimuth": 0, "lat": -26.029, "lon": -69.622, "tz": -4, "elev": 1600},
    "vallenar_simulado.csv": {"tilt": 28.6, "azimuth": 0, "lat": -28.576, "lon": -70.760, "tz": -4, "elev": 520},
}

# Layout
app.layout = dbc.Container([
    dbc.Row([dbc.Col(html.H1("Dashboard Solar PV", className="text-center my-4"), width=12)]),
    dbc.Row([
        dbc.Col([
            html.H4("Selección de Sitio"),
            dcc.Dropdown(
                id='site-dropdown',
                options=[{'label': k.replace('_simulado.csv', '').capitalize(), 'value': k} for k in site_params.keys()],
                value='salvador_simulado.csv'
            )
        ], width=4),
        dbc.Col([
            html.H4("Parámetros del Sistema"),
            dbc.Row([
                dbc.Col([html.Label("Capacidad AC (MW)"), dcc.Input(id='ac-capacity', type='number', value=50)], width=6),
                dbc.Col([html.Label("Ratio AC/DC"), dcc.Input(id='ac-dc-ratio', type='number', value=1.2)], width=6)
            ])
        ], width=8)
    ], className="mb-4"),

    dbc.Tabs([
        dbc.Tab(label="Análisis de Datos", children=[
            dbc.Row([
                dbc.Col([dcc.Graph(id='irradiance-profile')], width=6),
                dbc.Col([dcc.Graph(id='ghi-monthly')], width=6)
            ])
        ]),
        dbc.Tab(label="Simulación PV", children=[
            dbc.Row([
                dbc.Col([dcc.Graph(id='monthly-production')], width=6),
                dbc.Col([dcc.Graph(id='capacity-factor')], width=6)
            ])
        ]),
        dbc.Tab(label="Análisis Económico", children=[
            dbc.Row([
                dbc.Col([
                    html.H4("Parámetros Económicos"),
                    dbc.Row([
                        dbc.Col([html.Label("FCR (%)"), dcc.Input(id='fcr-input', type='number', value=8)], width=4),
                        dbc.Col([html.Label("CapEx (USD/kW)"), dcc.Input(id='capex-input', type='number', value=850)], width=4),
                        dbc.Col([html.Label("Precio Spot (USD/MWh)"), dcc.Input(id='spot-price', type='number', value=50)], width=4)
                    ])
                ], width=12)
            ]),
            dbc.Row([dbc.Col([dcc.Graph(id='lcoe-sensitivity')], width=12)])
        ]),
        dbc.Tab(label="KPI Diarios", children=[
            dbc.Row([
                dbc.Col(html.H5("Selecciona día típico (MM-DD):"), width=3),
                dbc.Col(dcc.DatePickerSingle(
                    id='day-selector',
                    display_format='MM-DD',
                    initial_visible_month='2004-01-01',
                    date='2004-01-01'
                ), width=3)
            ]),
            dbc.Row([dbc.Col([dcc.Graph(id='power-daily-profile')], width=12)]),
            dbc.Row([
                dbc.Col(dcc.Graph(id='kpi-energy'), width=4),
                dbc.Col(dcc.Graph(id='kpi-lcoe'), width=4),
                dbc.Col(dcc.Graph(id='kpi-cf'), width=4)
            ])
        ])
    ])
], fluid=True)

# Callbacks

@app.callback(
    [Output('irradiance-profile', 'figure'),
     Output('ghi-monthly', 'figure')],
    [Input('site-dropdown', 'value')]
)
def update_analysis_graphs(selected_site):
    df = pd.read_csv(selected_site)
    df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    df.set_index('datetime', inplace=True)

    irradiance_profile = go.Figure()
    for col in ['GHI', 'DNI', 'DHI']:
        irradiance_profile.add_trace(go.Scatter(
            x=df.groupby(df.index.hour)[col].mean().index,
            y=df.groupby(df.index.hour)[col].mean().values,
            name=col
        ))
    irradiance_profile.update_layout(
        title='Perfil Diario Promedio de Irradiancia',
        xaxis_title='Hora del día',
        yaxis_title='Irradiancia (W/m²)'
    )

    ghi_monthly_avg = df.groupby(df.index.month)['GHI'].mean()
    months = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    ghi_monthly = go.Figure(data=[go.Bar(x=months, y=ghi_monthly_avg, marker_color='teal')])
    ghi_monthly.update_layout(title='Promedio mensual de GHI', xaxis_title='Mes', yaxis_title='GHI promedio (W/m²)')

    return irradiance_profile, ghi_monthly

@app.callback(
    [Output('monthly-production', 'figure'),
     Output('capacity-factor', 'figure')],
    [Input('site-dropdown', 'value'),
     Input('ac-capacity', 'value'),
     Input('ac-dc-ratio', 'value')]
)
def update_pv_simulation(selected_site, ac_capacity, ac_dc_ratio):
    params = site_params[selected_site]
    df = pd.read_csv(selected_site)

    weather = {
        'year': df['Year'].tolist(), 'month': df['Month'].tolist(), 'day': df['Day'].tolist(),
        'hour': df['Hour'].tolist(), 'minute': df['Minute'].tolist(),
        'dn': df['DNI'].tolist(), 'df': df['DHI'].tolist(), 'gh': df['GHI'].tolist(),
        'tdry': df['Tdry'].fillna(25).tolist(), 'wspd': df['Wspd'].fillna(2).tolist(),
        'pres': df['Pres'].fillna(1013).tolist(), 'tz': params['tz'],
        'lat': params['lat'], 'lon': params['lon'], 'elev': params['elev']
    }

    sim = pv.default("PVWattsNone")
    sim.SystemDesign.system_capacity = ac_capacity * ac_dc_ratio * 1000
    sim.SystemDesign.dc_ac_ratio = ac_dc_ratio
    sim.SystemDesign.tilt = params['tilt']
    sim.SystemDesign.azimuth = params['azimuth']
    sim.SolarResource.solar_resource_data = weather
    sim.execute()

    monthly_prod = go.Figure(data=[go.Bar(
        x=['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic'],
        y=sim.Outputs.ac_monthly)])
    monthly_prod.update_layout(title='Producción Mensual de Energía (AC)', yaxis_title='Energía (kWh)')

    capacity_factor = go.Figure(data=[go.Indicator(
        mode="gauge+number", value=sim.Outputs.capacity_factor,
        title={'text': "Factor de Capacidad (%)"},
        gauge={'axis': {'range': [0, 30]}}
    )])

    return monthly_prod, capacity_factor

@app.callback(
    Output('lcoe-sensitivity', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('fcr-input', 'value'),
     Input('capex-input', 'value'),
     Input('spot-price', 'value')]
)
def update_lcoe_sensitivity(selected_site, fcr, capex, spot_price):
    df = pd.read_csv("produccion_anual_por_sitio.csv")
    sitio = selected_site.replace('_simulado.csv', '').capitalize()
    prod_anual = df[df['Sitio'] == sitio]['Produccion_Anual_kWh'].iloc[0]

    def calcular_lcoe(prod_kwh, fcr, capex, spot):
        capacidad_kw = 50000
        opex = 15 * capacidad_kw
        vida = 20
        energia_util = prod_kwh * (1 - 0.14)
        lcoe = (fcr * capex * capacidad_kw + opex * vida) / (energia_util * vida)
        return lcoe * 1000

    lcoe_base = calcular_lcoe(prod_anual, fcr/100, capex, spot_price)

    parametros = {
        "FCR": [fcr * 0.75, fcr * 1.25],
        "CapEx PV": [capex * 0.8, capex * 1.2],
        "Precio spot": [spot_price * 0.8, spot_price * 1.2]
    }

    fig = go.Figure()
    for param, (low, high) in parametros.items():
        if param == "FCR":
            l_low = calcular_lcoe(prod_anual, low/100, capex, spot_price)
            l_high = calcular_lcoe(prod_anual, high/100, capex, spot_price)
        elif param == "CapEx PV":
            l_low = calcular_lcoe(prod_anual, fcr/100, low, spot_price)
            l_high = calcular_lcoe(prod_anual, fcr/100, high, spot_price)
        else:
            l_low = calcular_lcoe(prod_anual, fcr/100, capex, low)
            l_high = calcular_lcoe(prod_anual, fcr/100, capex, high)

        fig.add_trace(go.Bar(y=[param], x=[l_high - lcoe_base], name='High', orientation='h', marker=dict(color='red')))
        fig.add_trace(go.Bar(y=[param], x=[l_low - lcoe_base], name='Low', orientation='h', marker=dict(color='green')))

    fig.add_vline(x=0, line_dash="dash", line_color="black")
    fig.update_layout(title=f"Análisis de Sensibilidad LCOE (Base: {lcoe_base:.2f} USD/MWh)",
                      xaxis_title="Variación del LCOE (USD/MWh)",
                      showlegend=False, barmode='overlay')
    return fig

@app.callback(
    Output('power-daily-profile', 'figure'),
    [Input('site-dropdown', 'value')]
)
def update_power_profile(site):
    df = pd.read_csv(site)
    df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    df.set_index('datetime', inplace=True)
    df['MM-DD'] = df.index.strftime('%m-%d')

    power_by_hour = df.groupby(['MM-DD', df.index.hour])['AC'].mean().unstack(level=0)
    ordered_mmdd = sorted(power_by_hour.columns, key=lambda x: datetime.strptime(x, "%m-%d"))
    power_by_hour = power_by_hour[ordered_mmdd]

    fig = go.Figure()
    for mmdd in power_by_hour.columns:
        fig.add_trace(go.Scatter(
            x=power_by_hour.index,
            y=power_by_hour[mmdd],
            name=mmdd,
            mode='lines',
            hovertemplate='Hora: %{x}<br>Potencia: %{y:.2f} kW<br>Día: ' + mmdd
        ))

    fig.update_layout(
        title="Curvas Horarias de Potencia AC",
        xaxis_title="Hora del día",
        yaxis_title="Potencia AC (kW)",
        hovermode='x unified'
    )
    return fig

@app.callback(
    [Output('kpi-energy', 'figure'),
     Output('kpi-lcoe', 'figure'),
     Output('kpi-cf', 'figure')],
    [Input('day-selector', 'date')],
    [State('site-dropdown', 'value'),
     State('ac-capacity', 'value')]
)
def update_kpis_por_mmdd(date_str, site, ac_capacity):
    if not date_str:
        return [go.Figure(), go.Figure(), go.Figure()]
    mmdd = pd.to_datetime(date_str).strftime('%m-%d')

    df = pd.read_csv(site)
    df['MM-DD'] = df['Month'].astype(str).str.zfill(2) + '-' + df['Day'].astype(str).str.zfill(2)
    df_dia = df[df['MM-DD'] == mmdd]

    if df_dia.empty or 'AC' not in df_dia.columns:
        return [go.Figure(), go.Figure(), go.Figure()]

    energia_kwh = (df_dia['AC'] / 1000).sum()
    cf = energia_kwh / (ac_capacity * 1000 * 24) * 100
    capex = 850
    fcr = 0.08
    opex = 15 * ac_capacity * 1000 / 365
    energia_util = energia_kwh * (1 - 0.14)
    lcoe = (fcr * capex * ac_capacity * 1000 / 365 + opex) / energia_util * 1000

    fig_energy = go.Figure(go.Indicator(mode="number+delta", value=energia_kwh,
        title={"text": f"Energía típica (kWh)<br><span style='font-size:0.8em'>{mmdd}</span>"}))

    fig_lcoe = go.Figure(go.Indicator(mode="number", value=lcoe,
        title={"text": "LCOE típico (USD/MWh)"}))

    fig_cf = go.Figure(go.Indicator(mode="gauge+number", value=cf,
        title={"text": "Factor de Capacidad Típico (%)"},
        gauge={'axis': {'range': [0, 30]}}))

    return fig_energy, fig_lcoe, fig_cf

if __name__ == '__main__':
    app.run(debug=True)
